In [1]:
#pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

In [2]:
import numpy as np
import timeit

In [3]:
import scipy.io as sio

In [4]:
mat = sio.loadmat('../data/25FV47.mat')
#mat = sio.loadmat('../data/80BAU3B.mat')

In [5]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Tue Oct 06 15:57:50 2009',
 '__version__': '1.0',
 '__globals__': [],
 'NAME': array(['NAME          25FV47'], dtype='<U20'),
 'BIG': array([[1.e+32]]),
 'A': <821x1876 sparse matrix of type '<class 'numpy.float64'>'
 	with 10705 stored elements in Compressed Sparse Column format>,
 'b': array([[ 0.00000000e+00],
        [ 2.90000000e+01],
        [ 6.00000000e+01],
        [ 7.30000000e+01],
        [ 7.70000000e+01],
        [ 2.70000000e+01],
        [ 4.40000000e+01],
        [ 4.00000000e+00],
        [ 2.30000000e+01],
        [ 4.40000000e+01],
        [ 1.64000000e+02],
        [ 3.10000000e+01],
        [ 7.90000000e+01],
        [ 1.90000000e+01],
        [ 1.00000000e+00],
        [ 4.00000000e+00],
        [ 8.60000000e+01],
        [ 8.80000000e+01],
        [ 2.00000000e+01],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00

In [6]:
A=mat['A']
A.shape

(821, 1876)

In [7]:
A

<821x1876 sparse matrix of type '<class 'numpy.float64'>'
	with 10705 stored elements in Compressed Sparse Column format>

In [8]:
b=mat['b']
len(b)

821

In [9]:
c=mat['c']
len(c)

1876

In [10]:
lbounds=mat['lbounds']
ubounds=mat['ubounds']
print(len(lbounds))
print(len(ubounds))

1876
1876


In [11]:
def generates_matrix(A, blow, bupp, c):
    """
    Genera la matriz bajo el supuesto de que se tiene ya capturado los datos de entrada. Genera el problema del tipo:
    `
    Al tipo
    Ax <= b
    Concantenando renglones de A y juntando los limites de las restricciones (multiplicando por -1 donde sea necesario)
    """
    nrow, ncol = A.shape
    A = A.todense()
    
    # Concatenar A con -A de forma rbind
    A_new = np.concatenate((A,-A), axis=0)
    #A_new = hstack((A,-A))
    
    # Concatenar A_new con c de formca rbind
    A_new = np.vstack([A_new, c.T])
    
    # Generar la matriz de pivoteo
    I = np.eye(nrow*2 + 1, ncol*2 + 2)
        
    # Crear nuevo vector de restricciones
    b_new = np.concatenate((blow,-bupp,np.array([[0]])), axis=0)
    
    # Concatenar A_new, b_new de forma cbind
    A_new = np.c_[A_new, I, b_new]
    
    return A_new

In [12]:
generates_matrix(A, b, b, c)

matrix([[   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0.,   29.],
        [   0.,    0.,    0., ...,    0.,    0.,   60.],
        ...,
        [  -0.,   -0.,   -0., ...,    0.,    0., -146.],
        [  -0.,   -0.,   -0., ...,    0.,    0., -247.],
        [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [13]:
def generates_matrix(A, b, c):
    """
    Genera la matriz bajo el supuesto de que se tiene ya capturado los datos de entrada. Genera el problema del tipo:
    `
    Al tipo
    Ax <= b
    Concantenando renglones de A y juntando los limites de las restricciones (multiplicando por -1 donde sea necesario)
    """
    nrow, ncol = A.shape
    A = A.todense()
        
    # Concatenar A_new con c de formca rbind
    A_new = np.vstack([A, -c.T])
    
    # Generar la matriz de pivoteo
    #I = np.eye(nrow + 1, ncol + 2)
    # Con el cambio
    I = np.eye(nrow + 1, ncol + 2)*0
        
    # Crear nuevo vector de restricciones
    b_new = np.concatenate((b,np.array([[0]])), axis=0)
    #b_new = np.concatenate((b,np.array([0])), axis=0) #prueba
    
    # Concatenar A_new, b_new de forma cbind
    A_new = np.c_[A_new, I, b_new]
    
    return A_new

In [14]:
A.shape

(821, 1876)

In [15]:
c_prueba = np.array([-3, -5])
A_prueba = np.array([[1, 0], [0, 2], [3, 2]])
b_prueba = np.array([[4, 12, 18]])
#generates_matrix(A_prueba, b_prueba, c_prueba)

In [16]:
generates_matrix(A, b, c)

matrix([[  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,  29.],
        [  0.,   0.,   0., ...,   0.,   0.,  60.],
        ...,
        [  0.,   0.,   0., ...,   0.,   0., 146.],
        [  0.,   0.,   0., ...,   0.,   0., 247.],
        [ -0.,  -0.,  -0., ...,   0.,   0.,   0.]])

# Prueba de paquetería `mex`

In [17]:
from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, \
find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot

from mex.simplex.problem_definition import add_cons, \
constrain, add_obj, obj, maxz, minz

In [18]:
matrix = generates_matrix(A, b, c)
matrix = np.array(matrix)
matrix

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,  29.],
       [  0.,   0.,   0., ...,   0.,   0.,  60.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0., 146.],
       [  0.,   0.,   0., ...,   0.,   0., 247.],
       [ -0.,  -0.,  -0., ...,   0.,   0.,   0.]])

In [19]:
min(matrix[:-1, -1])

-160.0

In [20]:
start = timeit.timeit()

problem_approx = maxz(matrix)

end = timeit.timeit()
print(end - start)


KeyboardInterrupt: 

Comenzó a correr a las $3:11:00$

In [ ]:
start = timeit.timeit()

problem_approx = minz(matrix)

end = timeit.timeit()
print(end - start)
